In [1]:
%load_ext autoreload
%autoreload 2

import os

while "notebooks" in os.getcwd():
    os.chdir("..")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch

from belt_nlp.model.bert_truncated import BertClassifierTruncated

# Example - Model BERT with truncation of longer texts

In this notebook we will show how to use basic methods `fit` and `predict` for the BERT model with truncating texts longer than 512 tokens.

## Load data - sample of IMDB reviews in english

In [ ]:
SAMPLE_DATA_PATH = "sample_data/sample_data_eng.csv"

In [ ]:
df = pd.read_csv(SAMPLE_DATA_PATH)
df

## Divide to train and test sets

In [ ]:
texts = df["sentence"].tolist()
labels = df["target"].tolist()
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

## Fit the model

In [ ]:
MODEL_PARAMS = {
    "batch_size": 8,
    "learning_rate": 5e-5,
    "epochs": 3,
}
model = BertClassifierTruncated(**MODEL_PARAMS, device="cuda:6")

In [ ]:
model.fit(X_train, y_train, epochs=3)

## Get predictions

In [ ]:
preds = model.predict_classes(X_test)

## Calculate model accuracy on the test data

In [ ]:
accurate = sum(preds == np.array(y_test).astype(bool))
accuracy = accurate / len(y_test)

print(f"Test accuracy: {accuracy}")